In [1]:
'Accrual Upload Tool'
'This program opens an excel file, then uploads data from that file to the Starbucks accrual portal'
'The program utilizes Selenium to automate internet browser functions'

'The program utilizes Selenium to automate internet browser functions'

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.by import By

from time import sleep

#For excel import and dataframes
import numpy as np
import pandas as pd
import openpyxl
import json
#update chromedriver if needed and save in the same folder as script. Also can direct python to where it is saved
#https://sites.google.com/a/chromium.org/chromedriver/home

In [2]:
#Function to open Chrome and navigate to accrual tool

def open_Chrome():
    options = Options()
    options.add_argument("start-maximized")
    options.add_argument('--no-sandbox')

    #Choose type of webdriver
    driver = webdriver.Chrome(options=options) #alternative: (executable_path='\path\to\chromedriver')
    driver.set_page_load_timeout(30) #so it doesn't time out until 30 seconds have passed
    
    #Open up the Accrual Entry Tool
    url = "my_url"
    driver.get(url)
    
    return driver

In [3]:
#Function to create custom click that will wait until an element is clickable. Retrying every half second until it works or reaches 10 seconds, after which it times out

def my_click_ID(element_selector, driver):
    wait = WebDriverWait(driver, 10, poll_frequency=1)
    seconds = 0
    while seconds < 10:
        try:
            element = driver.find_element_by_id(element_selector) #finds element in html
            driver.execute_script("arguments[0].scrollIntoView()", element) #Scroll into view so Selenium can click on the element
            element.click()
            break
        except:
            sleep(.25)
            seconds = seconds + .25
    if seconds == 10:
        print('Timeout')
        raise ValueError('Element not found or program timed out - Input manually or try again')
        
def my_click_xpath(element_selector, driver):
    wait = WebDriverWait(driver, 10, poll_frequency=1)
    seconds = 0
    while seconds < 10:    
        try:
            element = driver.find_element_by_xpath(element_selector)
            driver.execute_script("arguments[0].scrollIntoView()", element)
            sleep(.25) #Gives dropdown time to load and stop moving - more consistently choose correct dropdown option
            element.click()
            break
        except:
            sleep(.25)
            seconds = seconds + .25
    if seconds == 10:
        print('Timeout')
        raise ValueError('Element not found or program timed out - Input manually or try again')
        
            

In [14]:
#Function input data from Excel
#Must run Open_Chrome function first, so webpage is open. hen this function can be applied in a loop to enter records
#Submits accruals or returns an error and error location - causing no submission

def Accrual_Entry_Excel(driver):
    
    entry_error = 0
    error_type = ''
    
    #Open new record input form
    my_click_ID("AddNewRecord", driver)

    #Switch to new record input form
    driver.switch_to.frame("New_Record_Frame")

    #Open Legal Entity dropdown menu
    my_click_ID('LE_Drop', driver)
    #Choose the region
    my_click_xpath('//*[@id="LE_Drop"]/div/ul/li[.="my_region"]', driver)

    #Open Cost_Center dropdown menu
    my_click_ID('Cost_Center_Drop', driver)
    #Choose the correct RC
    CostCenter = Entry_CostCenter
    xpath = f'//*[@id="Cost_Center_Drop"]/div/ul/li[starts-with(text(),"{CostCenter}")]'
    my_click_xpath(xpath, driver)
       
    #Open Expense Account Dropdown
    my_click_ID('Acct_Drop', driver)
    #Choose the correct GL
    Acct = Entry_Acct
    xpath = f'//*[@id="Acct_Drop"]/div/ul/li[starts-with(text(),"{Acct}")]'
    my_click_xpath(xpath, driver)
    
    #Input Vendor
    vendor = Entry_Vendor
    Vendor_Input = driver.find_element_by_name("Vendor")
    Vendor_Input.send_keys(vendor)
    
    #Input Invoice Number
    invoice = Entry_Invoice
    Invoice_Input = driver.find_element_by_name("InvoiceNum")
    Invoice_Input.send_keys(invoice)
      
    #Input Accrual Amount
    accrual = Entry_Amount
    Accrual_Input = driver.find_element_by_name("AccrualAmount")
    Accrual_Input.send_keys(accrual)  

    #Input Description - this is before Month because otherwise you need to click out of a text input box to allow it to load for error checking
    description = Entry_Description
    Description_Input = driver.find_element_by_name("Description")
    Description_Input.send_keys(description)    

    #Open Month dropdown
    my_click_ID('Month_Drop', driver)
    #Choose the correct month
    month = Entry_Month
    xpath = f'//*[@id="Month_Drop"]/div/ul/li[.="{month}"]'
    my_click_xpath(xpath, driver)  
    
    sleep(.25) #quick pause to allow any loading - might be useless
    
    
    'ERROR CATCHING BLOCK--------------------------------------------------'
    
    #Check correct LE was clicked
    Check_LE = json.loads(driver.find_element_by_id('LE_ClientState').get_attribute('value'))
    Check_LE = Check_LE['value']
    if Check_LE != 'my_region':
        entry_error = 1
        error_type = error_type +' LE'
        'there is an error'
    else:
        pass 
    
    #Get attribute to check correct RC was clicked
    Check_CostCenter = json.loads(driver.find_element_by_id('CostCenter_ClientState').get_attribute('value'))
    Check_CostCenter = Check_CostCenter['value']
    if Check_CostCenter != CostCenter:
        entry_error = 1
        error_type = error_type + ' CostCenter'
        'there is an error'
    else:
        pass
    
    #Get attribute to check correct Account was clicked
    Check_Account = json.loads(driver.find_element_by_id('Acct_ClientState').get_attribute('value'))
    Check_Account = Check_Account['value']
    if Check_Account != Acct:
        entry_error = 1
        error_type = error_type + ' Account'
    else:
        pass  
    
    #Make sure a vendor was entered
    Check_Vendor = json.loads(driver.find_element_by_id('Vendor_ClientState').get_attribute('value'))
    Check_Vendor = Check_Vendor['validationText']
    if Check_Vendor != vendor:
        entry_error = 1
        error_type = error_type + ' Vendor'
    else:
        pass    
    Check_Invoice = json.loads(driver.find_element_by_id('InvoiceNum_ClientState').get_attribute('value'))
    Check_Invoice = Check_Invoice['validationText']
    if Check_Invoice != invoice:
        entry_error = 1
        error_type = error_type + ' Invoice'
    else:
        pass  
    
    #Make sure an amount was entered
    Check_Amount = json.loads(driver.find_element_by_id('AccrualAmount_ClientState').get_attribute('value'))
    Check_Amount = Check_Amount['validationText']
    if round(float(Check_Amount), 2) != round(float(accrual), 2):
        entry_error = 1
        error_type = error_type + ' Amount'
    else:
        pass    
    
    #Get attribute to check correct Month was clicked
    Check_Month = json.loads(driver.find_element_by_id('Month_ClientState').get_attribute('value'))
    Check_Month = Check_Month['value']
    if Check_Month != month:
        entry_error = 1
        error_type = error_type + ' Month'
    else:
        pass  
    
    #Make sure a description was entered
    Check_Description = json.loads(driver.find_element_by_id('Description_ClientState').get_attribute('value'))
    Check_Description = Check_Description['validationText']
    if Check_Description != description:
        entry_error = 1
    else:
        pass   
    'END ERROR CATCHING-----------------------------'
    
    
    #Submission or denial based on presence of errors
    if entry_error == 1: #IF there is an error window will be closed and error block in for loop will be executed
        driver.switch_to.default_content()
        Close_Accruals_Window = driver.find_element_by_class_name("CloseButton")
        Close_Accruals_Window.click()
    else: #If no errors accruals will be submitted or window will be closed - block out sections based on testing/production
        'Below is to submit accruals'
        #Submit Accrual
        submit_button = driver.find_element_by_xpath('//*[@id="Save_input"]')
        submit_button.click()
        driver.switch_to.default_content()
        'Below is to test for loop by closing entry instead of submitting'
        #Switches back to the main window. Allows selenium to find 'close' button on entry form
        #driver.switch_to.default_content()
        #Close_Accruals_Window = driver.find_element_by_class_name("CloseButton")
        #Close_Accruals_Window.click()
        
    return [entry_error, error_type]


In [5]:
'Import data from Excel'
source_df = pd.read_excel('Entry_Form.xlsx', engine='openpyxl')
#source_df

'Import data from Excel'

In [19]:
#Notes
    #Works by looping through each line of Excel doc and creating entries if column_1 is 'Accrue'
    #Entry variables are implicitly passed to the 'Accrual_Entry_Excel' selenium function
    #If statements are used extensively to catch errors
    #Try/except block catches general errors in selenium

df=source_df.copy()
n=-1 #Set to negative 1 so that loop can start at 0 when selecting rows in dataframe
web_driver = open_Chrome() #Activate the web driver
for row_status in df['column_1']:
        n = n+1
        if row_status == 'Accrue' or row_status == 'accrue':
            
            #Catch empty cells in RC & GL before they cause errors
            if np.isnan(df['CostCenter'][n]) == True or np.isnan(df['Account'][n]) == True:
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: CostCenter or Account Code Missing"
                continue                
            
            #Assign Variables
            Entry_CostCenter = str(int(df['CostCenter'][n]))
            Entry_Acct = str(int(df['Account'][n]))
            Entry_Amount = str(df['Amount'][n])
            Entry_Vendor = str(df['Vendor'][n])
            Entry_Invoice = str(df['Invoice_Num'][n])
            Entry_Description = str(df['Description'][n])
            Entry_Month = str(df['Month'][n]).upper()
            
            #Error Catching Block - explanations after each "if"
                #Continue will send it back to top of loop if it catches an error -- selenium script does not run --
            if len(Entry_Acct) != 6: #Protect against code choosing wrong Account. If less than 6, code will choose first Account that starts with matching numbers
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Account Code"
                continue
            elif len(Entry_CostCenter) !=6: #Protect against code choosing wrong Cost Center. Same as above
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: RC Code"
                continue
            elif len(Entry_Month) !=3 or Entry_Month == 'NAN': #Month must be 3 characters and capitalized
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Month should be 3 characters"
                continue
            elif len(df[(df['Invoice_Num'] == Entry_Invoice) & (df['CostCenter'] == df['CostCenter'][n])])>1: #Cannot have duplicate invoices in the same Cost Center
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Duplicate Inv# in RC. If using across multiple periods you will have to input manually"
                continue
            elif Entry_Invoice == 'nan': #Need an invoice number
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: No Inv#"
                continue                
            elif len(Entry_Vendor) < 1 or Entry_Vendor == 'nan': #Need to have a vendor
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: No Vendor"
                continue
            elif Entry_Description == 'nan': #Need an entry description
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: No Description"
                continue
            elif Entry_Amount == 'nan': #Need any entry amount
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: No Entry Amount"
                continue                
            else: #Move on in code
                pass
            try: #Make sure entry amount is a number and round to two decimals
                Entry_Amount = round(float(Entry_Amount),2)
                if Entry_Amount >= 500000: #Need to include an attachment for over $500k
                    df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Amount Greater Than $500k, needs documentation"
                    continue
                elif Entry_Amount <= 0: #Technically the inputs can be zero, but I am trying to catch errors. So I require a manual import for values less than zero.
                    df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Amount is negative or 0. Enter manually if not an error"    
                    continue  
                else:
                    Entry_Amount = str(Entry_Amount)
                    pass  
            except:
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Amount not a Number"
                continue
                
            #Run Selenium Script
            try: 
                Entry = Accrual_Entry_Excel(web_driver)
                #More error catching for when Entry Values do not equal what Selenium clicked - due to timing errors
                if Entry[0] == 1: #When there os an error Accrual_Entry_Excel returns 1 and does not submit the accrual
                    error_location = Entry[1]
                    df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: Program Input Data Incorrectly:" + error_location
                    continue
                else: #No problems, worked as expected
                    df.loc[df.index[n], 'column_1'] = "Accrued"
                    continue
            except:
                sleep(3) #if there is an error wait 3 seconds - allows time to see what happened
                web_driver.switch_to.default_content()
                Close_Accruals_Window = web_driver.find_element_by_class_name("CloseButton")
                Close_Accruals_Window.click()
                df.loc[df.index[n], 'column_1'] = "ERROR-Not Accrued: General Web Form Error - Including Duplicate Invoice w/in Period"
                #closes out the pop-up window if an error is encountered
        else:
            continue
        
web_driver.quit()
df.to_excel("Entry_Form.xlsx", index = False) #Save down output to original file